In [ ]:
import requests
from bs4 import BeautifulSoup as BS
import pandas as pd
import numpy as np
import concurrent.futures
import itertools


In [ ]:
url = requests.get('https://m2.ru/moskva/nedvizhimost/kupit-kvartiru/?pageNumber=1') #делаю запрос на сайт, ура, 200, ответ есть
url

<Response [200]>

In [ ]:
bs_url = BS(url.content, 'html.parser') #закидываю запрос о сайте в blended soup, для более комфортной работы далее

Далее я хочу собрать ссылки по страницам с квартирами, по которым я и буду собирать свой датасет, для того с каждой страницы собираю ссылки на страницу с квартирой

In [ ]:
bs_url.find('div', class_ = 'ClOfferSnippet__gallery').find('a').get('href')  #тут смотрю, всё ли нормально работает с 1 ссылкой (чтобы потом использовать этот код)

'https://m2.ru/moskva/nedvizhimost/prodazha-1-komnatnoi-kvartiri-38-5-m-10-22-etazh-dbe68544-45ee-41a5-9c7f-23a3f1bb47a4/'

In [ ]:
flats = bs_url.findAll('div', class_ = 'ClOfferSnippet__gallery') #собираю информацию по всем квартирам со страницы, заодно посмотрю сколько их на 1 странице находится
len(flats)

20

In [ ]:
flat_links_1_page = []
for flat in flats:
  flat_links_1_page.append(flat.find('a').get('href'))     #проверяю, собирает ли у меня информацию с 1 страницы

In [ ]:
flat_links_1_page  # убеждаюсь, что собирает

['https://m2.ru/moskva/nedvizhimost/prodazha-1-komnatnoi-kvartiri-38-5-m-10-22-etazh-dbe68544-45ee-41a5-9c7f-23a3f1bb47a4/',
 'https://m2.ru/moskva/nedvizhimost/prodazha-2-komnatnoi-kvartiri-69-9-m-29-30-etazh-ab3f268c-45a8-4479-a6a7-b1c6b7e8eb07/',
 'https://m2.ru/moskva/nedvizhimost/prodazha-1-komnatnoi-kvartiri-38-6-m-9-24-etazh-7a4259ba-3152-4bd2-89e6-9ec92c661571/',
 'https://m2.ru/moskva/nedvizhimost/prodazha-1-komnatnoi-kvartiri-32-4-m-14-33-etazh-a643445f-e7f6-4fd5-abff-d9014f64bcc6/',
 'https://m2.ru/moskva/nedvizhimost/prodazha-kvartiri-so-svobodnoi-planirovkoi-42-4-m-39-41-etazh-2aef1ab1-26be-4b84-91b1-d08e86c05adb/',
 'https://m2.ru/moskva/nedvizhimost/prodazha-1-komnatnoi-kvartiri-32-3-m-14-33-etazh-8bee3762-dc94-449e-80f2-1338a4315e24/',
 'https://m2.ru/moskva/nedvizhimost/prodazha-2-komnatnoi-kvartiri-42-m-3-9-etazh-81393f45-e792-4a32-8684-8a1ebeac3c37/',
 'https://m2.ru/moskva/nedvizhimost/prodazha-1-komnatnoi-kvartiri-36-1-m-7-33-etazh-f1a34406-f349-4640-9a62-54064a28b

In [ ]:
pages_quantity = np.arange(501)[1::]  #на сайте 500 страниц с объявлениями, адресация страниц начинается с 1

In [ ]:
flat_links_all = []
for i in range(len(pages_quantity)):
  url_i = requests.get('https://m2.ru/moskva/nedvizhimost/kupit-kvartiru/?pageNumber='+str(pages_quantity[i]))
  bs_url_i = BS(url_i.content, 'html.parser')
  flats_i = bs_url_i.findAll('div', class_ = 'ClOfferSnippet__gallery')
  for flat in flats_i:
    flat_links_all.append(flat.find('a').get('href'))

In [ ]:
all_links_for_flats= list(set(flat_links_all)) #чтобы не было повторений, перегоню это в множесвто, а потом опять в список, чтобы ушли повторяющиеся значения 

In [ ]:
len(all_links_for_flats) #смотрю, насколько меньше стало ссылок, несильно на много - это радует

9659

In [ ]:
with open("all_links_for_flats.txt", "w") as file:     #запишу список в текстовый файл, чтоб каждый раз не запускать код по-новому и не ждать 20 минут
    for link in all_links_for_flats:
          print(link, file=file)
file.close()
     

### Переходим к сбору информации по каждой из квартир

In [ ]:
bs_url_room.find('div', class_ = 'fonts-module__primary___2PNSt DescriptionCell__value')

<div class="fonts-module__primary___2PNSt DescriptionCell__value">Хороший</div>

In [ ]:
#все используемые в парсинге функции
def m2(link): #для площади пишу функцию, чтоб оставались только численные значения
  j = ''
  q = link.find_all('div', class_ = 'grid-module__cell___3uBi0')
  for string in q:
    if string.text[:16] == 'Площадь квартиры':
      j=string.text[16:]
  m2_mean = ''
  for item in j:
    item = item.replace(',', '.')
    if ((item.isnumeric() == True) & (item != '²')) or item == '.':
      m2_mean+= str(item)
  return(float(m2_mean))

def dist_to_subway(link, nes_dist):
  if len(link.find('div', class_='OfferRouteCardList').find_all('a', class_='SubwayStation__link')) == 0:
    return (np.nan)
  subv_all = link.find('div', class_='OfferRouteCardList').find_all('div', class_='colors-named-module__secondary___2WPH_ fonts-module__primary___2PNSt')
  list_time= []
  for subv in subv_all:
    if subv.text[::2].isnumeric() == True:
      list_time.append(int(subv.text[:3]))
    else:
      list_time.append(int(subv.text[:2]))
  if nes_dist == 'max':
    return(max(list_time))
  if nes_dist == 'min':
    return(min(list_time))

def dist_to_centr(link):
  k = link.find('div', class_ ='ClClickableAddress__links').find_all('span')[-1].text
  road__ = ''
  for item in k:
    item = item.replace(',', '.')
    if ((item.isnumeric() == True) & (item != '²')) or item == '.':
      road__+= str(item)
  return(float(road__))

In [ ]:
f = open('sample_data/all_links_for_flats.txt', 'r')

all_links = [line.strip() for line in f]

In [ ]:
f.close

<function TextIOWrapper.close()>

In [ ]:
all_links[12]

'https://m2.ru/moskva/nedvizhimost/prodazha-2-komnatnoi-kvartiri-55-8-m-18-21-etazh-dbb74678-c12a-4282-82ac-73bb6e888770/'

In [ ]:
url_room = requests.get('https://m2.ru/moskva/nedvizhimost/prodazha-2-komnatnoi-kvartiri-55-8-m-18-21-etazh-dbb74678-c12a-4282-82ac-73bb6e888770/') #requests.get(all_links[0])
bs_url_room = BS(url_room.content, 'html.parser')

In [ ]:
need_inf(bs_url_room)

{'Наименование объявления': 'Продажа 2-комнатной квартиры 55,8\xa0м², 18/21\xa0этаж',
 'Город': 'Москва',
 'Район': 'САО',
 'Количество станций метро рядом': 3,
 'Самый длогий путь до метро': 17,
 'Самый короткий путь до метро': 15,
 'Расттояние до центра': 7.8,
 'Площадь квартиры': 55.8,
 'Цена квартиры': '28824000'}

In [ ]:
url_room = requests.get('https://m2.ru/moskva/nedvizhimost/prodazha-1-komnatnoi-kvartiri-36-9-m-12-14-etazh-05cfcf9f-36f6-440b-986c-95af4027ffd3/') #requests.get(all_links[0])
bs_url_room = BS(url_room.content, 'html.parser')

In [72]:
#определя функцию для того, что мне искать на страничке
def need_inf(link, df):
  url_room = requests.get(link)
  bs_url_room = BS(url_room.content, 'html.parser')

  name = bs_url_room.find('h1', itemprop = 'name').text

  city = bs_url_room.find('a', class_ = 'ClClickableAddress__link').text
  district_ = (bs_url_room.find('div', class_ ='ClClickableAddress__links').find_all('span')[1].text)[:-1]

  how_many_subways = len(bs_url_room.find('div', class_='OfferRouteCardList').find_all('a', class_='SubwayStation__link'))
  subway_road_max = dist_to_subway(bs_url_room, "max")
  subway_road_min = dist_to_subway(bs_url_room, "min")
  road_to_centr = dist_to_centr(bs_url_room)

  m2_ = m2(bs_url_room)
  price_ = bs_url_room.find('span', itemprop = 'price').get('content')

  df.loc[ len(df.index )]= [name, city, district_ , how_many_subways, subway_road_max, subway_road_min, road_to_centr, m2_, price_]
  return(df)

In [ ]:
need_inf(bs_url_room)

In [ ]:
df_flats = pd.DataFrame(columns=['Наименование объявления', 'Город', "Район" , 'Количество станций метро рядом' , "Самый длогий путь до метро", "Самый короткий путь до метро", "Расттояние до центра", "Площадь квартиры", "Цена квартиры"])

In [75]:
for url in all_links[:50]:
  need_inf(url, df_flats)

In [76]:
df_flats

,Наименование объявления,Город,Район,Количество станций метро рядом,Самый длогий путь до метро,Самый короткий путь до метро,Расттояние до центра,Площадь квартиры,Цена квартиры
0,"Продажа квартиры-студии 26 м², 12/22 этаж",Москва,САО,3,30.0,16.0,13.5,26.0,10200000
1,"Продажа 2-комнатной квартиры 51 м², 13/24 этаж",Москва,СВАО,3,27.0,11.0,12.1,51.0,15068970
2,"Продажа 3-комнатной квартиры 76,5 м², 12/52 этаж",Москва,ЗАО,3,21.0,15.0,10.7,76.5,35800000
3,"Продажа 1-комнатной квартиры 36,9 м², 12/14 этаж",Москва,ЮВАО,3,15.0,10.0,5.8,36.9,12500000
4,"Продажа 2-комнатной квартиры 77,4 м², 32/77 этаж",Москва,ЦАО,3,12.0,6.0,5.5,77.4,63500000
5,"Продажа 3-комнатной квартиры 92,2 м², 3/13 этаж",Москва,СЗАО,3,30.0,13.0,13.9,92.2,27983253
6,"Продажа 3-комнатной квартиры 73,4 м², 28/41 этаж",Москва,ЗАО,3,25.0,15.0,11.1,73.4,25000000
7,"Продажа 1-комнатной квартиры 42,9 м², 10/18 этаж",Москва,Новая Москва,3,28.0,14.0,29.2,42.9,7950000
8,"Продажа 2-комнатной квартиры 78 м², 38/68 этаж",Москва,ЦАО,3,12.0,4.0,5.6,78.0,69750000
9,"Продажа 3-комнатной квартиры 73,3 м², 2/14 этаж",Москва,Новая Москва,3,79.0,68.0,29.8,73.3,14073600


In [86]:
df_flats.to_csv('flats_info.csv', index= False)

В этом разделе я просто смотрю, работает ли код правильно

In [ ]:
bs_url_room.find('h1', itemprop = 'name').text

'Продажа 2-комнатной квартиры 53,7\xa0м², 2/17\xa0этаж'

In [ ]:
bs_url_room.find('a', class_ = 'ClClickableAddress__link').text

'Москва'

In [ ]:
m2(bs_url_room)

136.918101214

In [ ]:
bs_url_room.find('span', itemprop = 'price').get('content')

'14800000'

In [ ]:
(bs_url_room.find('div', class_ ='ClClickableAddress__links').find_all('span')[1].text)[:-1]

'ВАО'

In [ ]:
len(bs_url_room.find('div', class_='OfferRouteCardList').find_all('a', class_='SubwayStation__link'))

3

In [ ]:
dist_to_subway(bs_url_room, "min")

6

In [ ]:
dist_to_centr(bs_url_room)

12.2